# Jungle Coach — Live (v4)

Description: import sys, os, time, threading, queue, pandas as pd, numpy as np, joblib


In [1]:
import sys, os, time, threading, queue, pandas as pd, numpy as np, joblib
import nest_asyncio; nest_asyncio.apply()
from collections import deque
from IPython.display import clear_output
sys.path.append(os.path.abspath(".."))
from util.live_client import poll_live_client
from util.audio import speak
from util.recommendations import rank_by_strategy, ACTIONS, recommend_addons, dynamic_risk_threshold, detect_wincon
from util.meta_adapt import contextual_weights
from util.maml_adapter import MamlWrapper
MODE = "ensemble"
RISK_PROFILE = "balanced"  # toggle: 'safe' | 'balanced' | 'flip' | 'auto'
WIN_CON_PROFILE = os.getenv('WIN_CON_PROFILE', 'balanced')  # toggle: 'split'|'pick'|'siege'|'objective'|'scaling'|'teamfight'
WIN_CON_AUTO = True  # toggle: auto-detect win-con from state
WIN_CON_CALLOUTS = True  # toggle: call out when win-con changes
os.environ['WIN_CON_PROFILE'] = WIN_CON_PROFILE
import json, os
THRESH = None
try:
    with open("../models/model_meta.json", 'r') as f:
        meta = json.load(f)
        base = float(meta.get('thresholds',{}).get('f1', 0.54))
        adj = {"safe": 0.04, "balanced": 0.0, "flip": -0.04}[RISK_PROFILE]
        THRESH = max(0.05, min(0.95, base + adj))
except Exception:
    THRESH = {"safe": 0.62, "balanced": 0.54, "flip": 0.48}[RISK_PROFILE]
THRESH_BASE = THRESH  # base threshold; dynamic risk adjusts per state when MODE is live
POLL_INTERVAL_S = 1.0  # how often to poll live client
HILITE_WINDOW_S = 3.0  # seconds to look back for skirmish detection
TOPK_PRINT = 5         # also print top-K actions before speaking
ROLL_WINDOW_S = 30.0   # rolling window for live features
HUD_MODE = True        # live-refresh top-K HUD in notebook output
# Exploration and uncertainty controls
EXPLORATION_MODE = os.getenv('EXPLORATION_MODE','none')  # 'none'|'epsilon'|'ucb'
EPSILON = float(os.getenv('EPSILON','0.0'))
UCB_BETA = float(os.getenv('UCB_BETA','0.15'))  # multiply stddev bonus
EXPLORE_TOPK = int(os.getenv('EXPLORE_TOPK','5'))
UNCERTAINTY_ABSTAIN = bool(int(os.getenv('UNCERTAINTY_ABSTAIN','0')))
DISAGREE_THRESH = float(os.getenv('DISAGREE_THRESH','0.06'))  # stddev threshold on ensemble scores
ENSEMBLE_WEIGHT_MODE = os.getenv('ENSEMBLE_WEIGHT_MODE', 'meta')
MANUAL_STRAT_WEIGHTS = {"classification":1.0, "regression_ev":1.0, "rl_awr":1.0, "imitation":1.0, "maml":1.0}
STRAT_WEIGHTS = {"classification":1.0, "regression_ev":1.0, "rl_awr":1.0, "imitation":1.0, "maml":1.0}
if ENSEMBLE_WEIGHT_MODE == 'equal':
    STRAT_WEIGHTS = {k:1.0 for k in STRAT_WEIGHTS}
elif ENSEMBLE_WEIGHT_MODE == 'manual':
    STRAT_WEIGHTS.update(MANUAL_STRAT_WEIGHTS)
else:  # 'auc'
    try:
        with open("../models/model_meta.json", 'r') as f:
            auc = float((json.load(f) or {}).get('metrics',{}).get('auc', 0.65) or 0.65)
            STRAT_WEIGHTS['classification'] = 1.0 + max(0.0, auc-0.5)
    except Exception: pass
    try:
        with open("../models/model_meta_ev.json", 'r') as f:
            auc = float((json.load(f) or {}).get('metrics',{}).get('auc', 0.62) or 0.62)
            STRAT_WEIGHTS['regression_ev'] = 1.0 + max(0.0, auc-0.5)
    except Exception: pass
    try:
        with open("../models/model_meta_rl.json", 'r') as f:
            auc = float((json.load(f) or {}).get('metrics',{}).get('auc', 0.6) or 0.6)
            STRAT_WEIGHTS['rl_awr'] = 1.0 + max(0.0, auc-0.5)
    except Exception: pass
    try:
        with open("../models/model_meta_il.json", 'r') as f:
            acc = float((json.load(f) or {}).get('metrics',{}).get('acc_top1', 0.55) or 0.55)
            STRAT_WEIGHTS['imitation'] = 1.0 + max(0.0, acc-0.5)
    except Exception: pass
print("Mode:", MODE, "Risk:", RISK_PROFILE, "Thresh:", THRESH, "Weights:", STRAT_WEIGHTS)


Mode: ensemble Risk: balanced Thresh: 0.46581465005874634 Weights: {'classification': 1.3457997912636055, 'regression_ev': 1.0, 'rl_awr': 1.0}


Description: paths = {


In [2]:
paths = {
    "classification": "../models/jungle_bc.joblib",
    "regression_ev": "../models/ev_multi.joblib",
    "ev_win_mapper": "../models/ev_win_mapper.joblib",
    "rl_awr": "../models/rl_policy.joblib",
    "imitation": "../models/jungle_il.joblib",
    "rf": "../models/jungle_rf.joblib",
    "linear_cv": "../models/jungle_linear_cv.joblib",
    "actor_critic": "../models/actor_critic.joblib",
    "q_table": "../models/q_table.joblib",
    "stacking": "../models/stacking_meta.joblib",
}
models = {}
maml = None
for k,p in paths.items():
    if os.path.exists(p):
        try:
            models[k] = joblib.load(p)
            print("Loaded:", k, "->", p)
        except Exception as e:
            print("Failed to load", k, ":", e)
if "ev_win_mapper" in models and "regression_ev" not in models:
    models.pop("ev_win_mapper", None)
# Initialize MAML wrapper if artifact exists
try:
    if os.path.exists('../models/maml_cls.pt'):
        maml = MamlWrapper('../models/maml_cls.pt', inner_steps=1, lr_inner=1e-2)
        print('Loaded: MAML -> ../models/maml_cls.pt')
except Exception as e:
    print('MAML init failed:', e)


Loaded: classification -> ../models/jungle_bc.joblib
Loaded: regression_ev -> ../models/ev_multi.joblib
Loaded: ev_win_mapper -> ../models/ev_win_mapper.joblib
Loaded: rl_awr -> ../models/rl_policy.joblib


Description: state = {"prev": None, "hist": deque()}


In [3]:
state = {"prev": None, "hist": deque(), "maml_support": deque(maxlen=128)}
def features_from_live(allj: dict, events: list) -> pd.DataFrame:
    t = float((allj or {}).get("gameData",{}).get("gameTime", 0.0))
    players = (allj or {}).get("allPlayers", []) or []
    kills = {"ORDER":0,"CHAOS":0}
    wards_inv = {"ORDER":0,"CHAOS":0}
    has_herald = 0
    ally_support_engage = 0
    team_gold = {"ORDER":0.0, "CHAOS":0.0}
    team_level = {"ORDER":[], "CHAOS":[]}
    team_cs = {"ORDER":0, "CHAOS":0}
    ally_dead = 0; enemy_dead = 0
    ally_players = [p for p in players if p.get("team") == "ORDER"]
    support_cand = None
    if ally_players:
        support_cand = min(ally_players, key=lambda q: int(((q.get("scores") or {}).get("creepScore") or 0)))
    ENGAGE_SUPPORTS = {"Leona","Nautilus","Rell","Alistar","Rakan","Thresh","Blitzcrank","Braum","Pyke"}
    if support_cand and (support_cand.get("championName") in ENGAGE_SUPPORTS):
        ally_support_engage = 1
    # Champion-aware tags
    SPLIT = {"Fiora","Camille","Jax","Tryndamere","Yorick","Nasus"}
    POKE = {"Ziggs","Jayce","Xerath","Vel'Koz","Varus","Ezreal"}
    ENGAGE = {"Malphite","Sejuani","Jarvan IV","Amumu","Rell","Nautilus","Leona","Wukong","Zac"}
    ally_split_count = ally_poke_count = ally_engage_count = 0
    for p in players:
        tm = p.get("team")
        sc = p.get("scores",{})
        ch = p.get("championName")
        if ch:
            if tm == "ORDER":
                if ch in SPLIT: ally_split_count += 1
                if ch in POKE: ally_poke_count += 1
                if ch in ENGAGE: ally_engage_count += 1
        if bool(p.get("isDead", False)):
            if tm == "ORDER": ally_dead += 1
            elif tm == "CHAOS": enemy_dead += 1
        kills[tm] += int(sc.get("kills",0))
        try: team_gold[tm] += float(p.get("totalGold",0.0) or 0.0)
        except Exception: pass
        try: team_level[tm].append(int(p.get("level",0) or 0))
        except Exception: pass
        try: team_cs[tm] += int((sc.get("creepScore") or 0))
        except Exception: pass
        for it in (p.get("items",[]) or []):
            nm = (it.get("displayName") or "").lower()
            if "ward" in nm or "oracle" in nm or "sweeper" in nm:
                wards_inv[tm] += 1
            if "herald" in nm:
                has_herald = 1 if tm == "ORDER" else has_herald
    prev = state.get("prev")
    dk = 0
    if prev and abs(t - prev.get("t",0)) <= HILITE_WINDOW_S:
        dk = (kills["ORDER"]+kills["CHAOS"]) - (prev.get("kills_order",0)+prev.get("kills_chaos",0))
    skirm = 1 if dk>0 else 0
    vision_delta = wards_inv["ORDER"] - wards_inv["CHAOS"]
    hist = state.get("hist")
    hist.append((t, int(dk), int(vision_delta)))
    while hist and (t - hist[0][0]) > ROLL_WINDOW_S:
        hist.popleft()
    roll_recent_kills = sum(1 for _,dkv,_ in hist if dkv>0)
    roll_vision_delta = sum(v for *_,v in hist)
    phase = 1 if t < 8*60 else 2 if t < 14*60 else 3 if t < 20*60 else 4 if t < 30*60 else 5
    gold_diff = float(team_gold["ORDER"]) - float(team_gold["CHAOS"])
    lvl_o = (sum(team_level["ORDER"]) / max(1,len(team_level["ORDER"])) if team_level["ORDER"] else 0)
    lvl_c = (sum(team_level["CHAOS"]) / max(1,len(team_level["CHAOS"])) if team_level["CHAOS"] else 0)
    level_diff = float(lvl_o - lvl_c)
    cs_diff = int(team_cs["ORDER"]) - int(team_cs["CHAOS"])
    row = dict(
        time_s=t, dragon_diff=0, baron_diff=0, herald_diff=0, tower_diff=0, plate_diff=0, ward_kill_diff=0,
        t1_top_diff=0, t1_mid_diff=0, t1_bot_diff=0, t2_top_diff=0, t2_mid_diff=0, t2_bot_diff=0,
        inh_top_diff=0, inh_mid_diff=0, inh_bot_diff=0,
        team_kills_d=max(dk,0), team_deaths_d=0,
        skirmish_flag=int(skirm),
        plates_time_left=max(0, 14*60 - int(t)),
        phase_num=phase, baron_live=int(t>=20*60), dragon_live=1,
        vision_delta=vision_delta,
        has_herald=int(has_herald),
        ally_support_engage=int(ally_support_engage),
        team_gold_diff=gold_diff,
        team_level_avg_diff=level_diff,
        team_cs_diff=cs_diff,
        ally_dead_count=int(ally_dead), enemy_dead_count=int(enemy_dead), dead_diff=int(ally_dead-enemy_dead),
        ally_split_count=int(ally_split_count), ally_poke_count=int(ally_poke_count), ally_engage_count=int(ally_engage_count),
        team_gold_diff_per_min=float(60.0*gold_diff/max(1.0,t)),
        team_cs_diff_per_min=float(60.0*cs_diff/max(1.0,t)),
        flip_risk_proxy=0.0, setup_quality_score=0.0, recall_sync_score=0.0, crossmap_ev_proxy=0.0, smite_diff_proxy=0.0,
        roll_recent_kills_30s=int(roll_recent_kills), roll_vision_delta_30s=int(roll_vision_delta),
    )
    state["prev"] = {"t":t, "kills_order":kills["ORDER"], "kills_chaos":kills["CHAOS"]}
    return pd.DataFrame([row])

def speak_action(action: str, score: float, why: str):
    phrases = {
        "SETUP_DRAGON_90": "Set up dragon in ninety seconds: get vision and push mid.",
        "TAKE_DRAGON": "Secure dragon now if safe.",
        "SETUP_HERALD_60": "Set up Herald in one minute. Push top-side and get river wards.",
        "TAKE_HERALD": "Take Herald now if uncontested.",
        "SETUP_BARON_120": "Two minutes to Baron—establish vision top side.",
        "TAKE_BARON": "Start Baron if uncontested. Be ready to peel.",
        "PRESS_TOWER_MID_T1": "Press mid tier one.",
        "PRESS_TOWER_TOP_T1": "Pressure top tier one.",
        "PRESS_TOWER_BOT_T1": "Pressure bot tier one.",
        "PRESS_TOWER_TOP_T2": "Pressure top tier two.",
        "PRESS_TOWER_MID_T2": "Pressure mid tier two.",
        "PRESS_TOWER_BOT_T2": "Pressure bot tier two.",
        "PRESS_INHIB_TOP": "Look to open top inhibitor.",
        "PRESS_INHIB_MID": "Look to open mid inhibitor.",
        "PRESS_INHIB_BOT": "Look to open bot inhibitor.",
        "LOOK_FOR_PICK": "Look for a pick with your teammates.",
        "JOIN_RIVER_FIGHT": "Join the river fight.",
        "COVER_COUNTERGANK": "Cover lanes for potential counterganks.",
        "FARM_BLUE": "Take your blue.",
        "FARM_GROMP": "Take gromp.",
        "FARM_WOLVES": "Take wolves.",
        "FARM_RAPTORS": "Take raptors.",
        "FARM_RED": "Take red.",
        "FARM_KRUGS": "Take krugs.",
        "SECURE_SCUTTLE_TOP": "Secure top scuttle.",
        "SECURE_SCUTTLE_BOT": "Secure bot scuttle.",
        "CLEAR_TOP_CAMPS": "Clear your top-side camps.",
        "CLEAR_BOT_CAMPS": "Clear your bot-side camps.",
        "INVADE_TOP_CAMPS": "Invade top-side jungle; place deep wards.",
        "INVADE_BOT_CAMPS": "Invade bot-side jungle; place deep wards.",
        "JOIN_FIGHT_TOP": "Join fight top lane.",
        "JOIN_FIGHT_MID": "Join fight mid lane.",
        "JOIN_FIGHT_BOT": "Join fight bot lane.",
        "JOIN_FIGHT_TOP_JUNGLE": "Join fight top jungle.",
        "JOIN_FIGHT_BOT_JUNGLE": "Join fight bot jungle.",
        "USE_HERALD": "Drop Herald to take plates or open tower.",
        "GANK_TOP": "Look to gank top.",
        "GANK_MID": "Look to gank mid.",
        "GANK_BOT": "Look to gank bot.",
        "RESET_BUY": "Reset to buy and set up vision.",
        "DEEP_VISION_SWEEP": "Sweep deep vision and control entrances.",
        "PING_PRESS_SIDES": "Ping: press sides and draw pressure.",
        "PING_GROUP_OBJECTIVE": "Ping: group for objective.",
        "PING_GROUP_BARON": "Ping: group top river for Baron.",
        "PING_GROUP_MID": "Ping: group mid to fight.",
        "PING_POKE_SETUP": "Ping: set up poke and chip before commit.",
        "PING_LOOK_FOR_PICK": "Ping: look for a pick together.",
        "PING_SAFE_SCALE": "Ping: play safe and scale.",
    }
    msg = f"{phrases.get(action, action)}  Confidence {score:.2f}."
    print(msg, ("— " + why if why else ""))
    return msg


Description: stop = threading.Event()


In [5]:
stop = threading.Event()
q = queue.Queue()
th = threading.Thread(target=poll_live_client, args=(stop,q,POLL_INTERVAL_S), daemon=True)
th.start()
print("Connecting to Live Client… (start a game; interrupt to stop)")

try:
    while True:
        pkt = q.get()
        allj, events = pkt.get("all"), pkt.get("events")
        if not allj:
            print("Live Client API not reachable. Start a LoL game locally.")
            continue
        feats = features_from_live(allj, events)
        # Win-con detection and callout
        try:
            _state_row = feats.iloc[0].to_dict()
            if WIN_CON_AUTO:
                _wc = detect_wincon(_state_row)
            else:
                _wc = WIN_CON_PROFILE
            # publish as env for weighting
            os.environ['WIN_CON_PROFILE'] = _wc
            # callout if changed
            _prev_wc = state.get('wincon_last') if isinstance(state, dict) else None
            if WIN_CON_CALLOUTS and (_wc != _prev_wc):
                print(f"Win-con changed: {_wc}")
                speak(f"Win condition: {_wc}.")
                state['wincon_last'] = _wc
        except Exception:
            pass
        recs_all = []
        if MODE in ("classification","ensemble") and os.path.exists("../models/jungle_bc.joblib"):
            clf = joblib.load("../models/jungle_bc.joblib")
            recs_all.append(("classification", rank_by_strategy("classification", clf, feats)))
            # Pseudo-labels for MAML support buffer from high-confidence classifier preds
            try:
                _p = float(clf.predict_proba(feats)[:,1][0])
                if _p >= 0.9 or _p <= 0.1:
                    state['maml_support'].append({'x': feats.copy(), 'y': (1.0 if _p>=0.9 else 0.0)})
            except Exception:
                pass
        if MODE in ("regression_ev","ensemble") and os.path.exists("../models/ev_multi.joblib") and os.path.exists("../models/ev_win_mapper.joblib"):
            ev = joblib.load("../models/ev_multi.joblib")
            mapper = joblib.load("../models/ev_win_mapper.joblib")
            recs_all.append(("regression_ev", rank_by_strategy("regression_ev", ev, feats, ev_mapper=mapper)))
        if MODE in ("rl_awr","ensemble") and os.path.exists("../models/rl_policy.joblib"):
            pi = joblib.load("../models/rl_policy.joblib")
            recs_all.append(("rl_awr", rank_by_strategy("rl_awr", pi, feats)))
        if MODE in ("rf","ensemble") and os.path.exists("../models/jungle_rf.joblib"):
            rf = joblib.load("../models/jungle_rf.joblib")
            recs_all.append(("rf", rank_by_strategy("rf", rf, feats)))
        if MODE in ("linear_cv","ensemble") and os.path.exists("../models/jungle_linear_cv.joblib"):
            lcv = joblib.load("../models/jungle_linear_cv.joblib")
            recs_all.append(("linear_cv", rank_by_strategy("linear_cv", lcv, feats)))
        if MODE in ("imitation","ensemble") and os.path.exists("../models/jungle_il.joblib"):
            il = joblib.load("../models/jungle_il.joblib")
            recs_all.append(("imitation", rank_by_strategy("imitation", il, feats)))
        if MODE in ("actor_critic","ensemble") and os.path.exists("../models/actor_critic.joblib"):
            ac = joblib.load("../models/actor_critic.joblib")
            try:
                pol = ac.get("policy") if isinstance(ac, dict) else ac
            except Exception:
                pol = ac
            recs_all.append(("actor_critic", rank_by_strategy("imitation", pol, feats)))
        if MODE in ("q_table","ensemble") and os.path.exists("../models/q_table.joblib"):
            pack = joblib.load("../models/q_table.joblib")
            from util.recommendations import ACTIONS
            # Simple Q-based ranking: normalize Q(s, a) over candidates
            row = feats.iloc[0].to_dict()
            from training.train_q_learning import _make_state
            s = _make_state(row)
            Q = pack.get("Q", {})
            cand = [a for a in ACTIONS]
            vals = []
            for a in cand:
                vals.append((a, float(Q.get((s,a), 0.0))))
            vals.sort(key=lambda x: x[1], reverse=True)
            recs_all.append(("q_table", [(a, 0.5 + 0.5*v if not np.isnan(v) else 0.5, "q") for a,v in vals]))
        if MODE in ("maml","ensemble") and maml is not None:
            try:
                pm = float(maml.predict_proba(feats, support=list(state.get('maml_support') or []))[0,1])
                class _Shim:
                    def __init__(self, p): self._p=p
                    def predict_proba(self, _):
                        import numpy as _np
                        return _np.array([[1.0-self._p, self._p]], dtype=float)
                recs_all.append(("maml", rank_by_strategy("stacking", _Shim(pm), feats)))
            except Exception:
                pass
        if not recs_all:
            continue
        import numpy as np
        tally = {}
        # Meta-adaptive weighting: if ENSEMBLE_WEIGHT_MODE == 'meta', re-weight per state
        w_map = STRAT_WEIGHTS
        try:
            if ENSEMBLE_WEIGHT_MODE == 'meta':
                w_map = contextual_weights(feats.iloc[0].to_dict(), base=STRAT_WEIGHTS)
        except Exception:
            w_map = STRAT_WEIGHTS
        for name, lst in recs_all:
            w_mult = float(w_map.get(name, 1.0))
            for rank, (a,score,why) in enumerate(lst[:10], start=1):
                w = (11-rank) * w_mult
                tally.setdefault(a, {"votes":0,"scores":[],"whys":[]})
                tally[a]["votes"] += w
                tally[a]["scores"].append(score)
                if why: tally[a]["whys"].append(f"{name}:{why}")
        sorted_items = sorted(tally.items(), key=lambda kv: (kv[1]['votes'], np.mean(kv[1]['scores'])), reverse=True)
        # Optional exploration: epsilon-greedy or UCB-like using stddev as bonus
        pick = 0
        if EXPLORATION_MODE == 'epsilon':
            try:
                if np.random.rand() < EPSILON:
                    pick = int(min(len(sorted_items)-1, max(0, EXPLORE_TOPK-1)))
                    pick = int(np.random.randint(0, pick+1))
            except Exception:
                pick = 0
        elif EXPLORATION_MODE == 'ucb':
            try:
                def _ucb(item):
                    v = item[1]
                    m = float(np.mean(v['scores']))
                    s = float(np.std(v['scores']))
                    return m + UCB_BETA * s
                ucb_sorted = sorted(sorted_items, key=_ucb, reverse=True)
                sorted_items = ucb_sorted
            except Exception:
                pass
        best = sorted_items[pick]
        action = best[0]; score = float(np.mean(best[1]['scores']))
        # Uncertainty gating: if ensemble disagrees, switch to safe utility action
        try:
            sd = float(np.std(best[1]['scores']))
        except Exception:
            sd = 0.0
        if UNCERTAINTY_ABSTAIN and sd >= DISAGREE_THRESH:
            # Prefer a safe, information-gathering action
            if 'DEEP_VISION_SWEEP' in tally:
                action = 'DEEP_VISION_SWEEP'
                score = float(np.mean(tally[action]['scores'])) if tally.get(action) else score
                why = 'uncertainty_abstain'
            else:
                action = 'RESET_BUY'
                score = float(np.mean(tally[action]['scores'])) if tally.get(action) else score
                why = 'uncertainty_abstain'
        why = ";".join(best[1]["whys"][:2])
        # Dynamic risk: compute threshold per game state and phase
        try:
            dyn_thresh, risk_lbl = dynamic_risk_threshold(feats.iloc[0].to_dict(), THRESH_BASE, profile=RISK_PROFILE if RISK_PROFILE!='balanced' else 'auto')
        except Exception:
            dyn_thresh, risk_lbl = THRESH_BASE, RISK_PROFILE
        try:
            addons = recommend_addons(feats.iloc[0].to_dict(), include_warnings=True, action=action)
        except Exception:
            addons = []
        topk = sorted_items[:TOPK_PRINT] if TOPK_PRINT and TOPK_PRINT>0 else []
        if HUD_MODE:
            clear_output(wait=True)
            print("Top actions (votes, avg score):")
            print(f"Win-con: {os.getenv('WIN_CON_PROFILE','balanced')}")
            for k,v in topk:
                print(f" - {k}: {v['votes']} votes, {np.mean(v['scores']):.3f}")
            print(f"Risk mode: {risk_lbl} | dyn_thresh={dyn_thresh:.2f}")
            if addons:
                print("Add-ons: " + ", ".join([f"{a[0]} ({a[1]:.2f})" for a in addons[:3]]))
                warn = next((a for a in addons if a[0]=="WARNINGS" and a[2]), None)
                if warn:
                    print("Warnings:", warn[2])
        msg = speak_action(action, score, why)
        if score >= dyn_thresh:
            speak(msg)
except KeyboardInterrupt:
    stop.set(); th.join(timeout=2.0); print("Stopped.")


Connecting to Live Client… (start a game; interrupt to stop)
Live Client API not reachable. Start a LoL game locally.
Live Client API not reachable. Start a LoL game locally.
Live Client API not reachable. Start a LoL game locally.
Live Client API not reachable. Start a LoL game locally.
Stopped.
